import pandas as pd

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt


In [49]:
df = pd.read_csv('listings.csv')
print (df.shape)
df.head ()

(76534, 16)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,13913,Holiday London DB Room Let-on going,54730,Alina,NaN,Islington,51.56802,-0.11121,Private room,40,1,21,22/02/2020,0.16,2,365
1,15400,Bright Chelsea Apartment. Chelsea!,60302,Philippa,NaN,Kensington and Chelsea,51.48796,-0.16898,Entire home/apt,75,10,89,16/03/2020,0.66,1,189
2,17402,Superb 3-Bed/2 Bath & Wifi: Trendy W1,67564,Liz,NaN,Westminster,51.52195,-0.14094,Entire home/apt,307,4,42,02/11/2019,0.35,15,270
3,17506,Boutique Chelsea/Fulham Double bed 5-star ensuite,67915,Charlotte,NaN,Hammersmith and Fulham,51.47935,-0.19743,Private room,150,3,0,NaN,NaN,2,364
4,25123,Clean big Room in London (Room 1),103583,Grace,NaN,Barnet,51.57224,-0.20906,Private room,29,10,129,15/03/2020,0.98,3,43


In [50]:
#viewing every columns properly
count = 1
for i in df. columns:
    print (count, i)
    count += 1

1 id
2 name
3 host_id
4 host_name
5 neighbourhood_group
6 neighbourhood
7 latitude
8 longitude
9 room_type
10 price
11 minimum_nights
12 number_of_reviews
13 last_review
14 reviews_per_month
15 calculated_host_listings_count
16 availability_365


In [51]:
#1. Take only the columns that are necessary for analysis
# No of reviews datas was found misleading so it has not been considered here
df = df [['id', 'neighbourhood', 'name', 'room_type', 'price']]
#df = f[['id', 'description', 'zipcode']]
df


,id,neighbourhood,name,room_type,price
0,13913,Islington,Holiday London DB Room Let-on going,Private room,40
1,15400,Kensington and Chelsea,Bright Chelsea Apartment. Chelsea!,Entire home/apt,75
2,17402,Westminster,Superb 3-Bed/2 Bath & Wifi: Trendy W1,Entire home/apt,307
3,17506,Hammersmith and Fulham,Boutique Chelsea/Fulham Double bed 5-star ensuite,Private room,150
4,25123,Barnet,Clean big Room in London (Room 1),Private room,29
...,...,...,...,...,...
76529,48101619,Westminster,One Bedroom Apartment Mayfair,Entire home/apt,105
76530,48102439,Lambeth,Amazing Clapham Common 4bedroom house,Entire home/apt,90
76531,48102612,Wandsworth,Luxury Manhattan 2 Bed Battersea,Entire home/apt,139
76532,48102679,Camden,GuestReady - Lux Central 2BR Garden Flat in Fi...,Entire home/apt,177


In [52]:
#Check for any NaN values that affect the dataset
print('Number of rows in each column affected by existence of non-existing values:')
df.isnull().sum()

Number of rows in each column affected by existence of non-existing values:


id                0
neighbourhood     0
name             24
room_type         0
price             0
dtype: int64

In [53]:
df = df.dropna()
df.isnull().sum()

id               0
neighbourhood    0
name             0
room_type        0
price            0
dtype: int64

In [56]:
review = pd.read_csv('reviews.csv')
review.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,13913,80770,2010-08-18,177109,Michael,My girlfriend and I hadn't known Alina before ...
1,13913,367568,2011-07-11,19835707,Mathias,Alina was a really good host. The flat is clea...
2,13913,529579,2011-09-13,1110304,Kristin,Alina is an amazing host. She made me feel rig...
3,13913,595481,2011-10-03,1216358,Camilla,"Alina's place is so nice, the room is big and ..."
4,13913,612947,2011-10-09,490840,Jorik,"Nice location in Islington area, good for shor..."


In [57]:
review = review[['listing_id', 'comments']]

In [62]:
#1. Remove rows that do not contain comments.
reviewNAcomments = review[(review.comments.isnull())]
print(reviewNAcomments.shape)
review=review[~(review.comments.isnull())]

(512, 2)


In [63]:
#group the dataframe by Listing id and then bring all the comments to a particular listing_id to group accord
review_group = review.groupby('listing_id')
print(review_group)
review = review_group.apply(lambda x: list(x[ 'comments']))

In [64]:
#Convert from series to dataframe
review = review.to_frame ('comments')
review

,comments
listing_id,
13913,[My girlfriend and I hadn't known Alina before...
15400,[We loved staying at Phillipa's place in Chels...
17402,"[Amazing location - great apartment, plenty of..."
25123,"[10,000 times better and cheaper than staying ..."
33332,"[The room is spacious and cozy, and the house ..."
...,...
47948833,[Fantastic hospitality]
47957224,[Salahudin was a great host! He was extremely ...
47975947,"[Great place, very pleasent, great host, Cosy ..."


In [65]:
# merging full review + add only specific columns from df
cleaned_df = pd.merge(left=review, right=df, how='left', left_on=review.index, right_on='id')
cleaned_df

,comments,id,neighbourhood,name,room_type,price
0,[My girlfriend and I hadn't known Alina before...,13913,Islington,Holiday London DB Room Let-on going,Private room,40.0
1,[We loved staying at Phillipa's place in Chels...,15400,Kensington and Chelsea,Bright Chelsea Apartment. Chelsea!,Entire home/apt,75.0
2,"[Amazing location - great apartment, plenty of...",17402,Westminster,Superb 3-Bed/2 Bath & Wifi: Trendy W1,Entire home/apt,307.0
3,"[10,000 times better and cheaper than staying ...",25123,Barnet,Clean big Room in London (Room 1),Private room,29.0
4,"[The room is spacious and cozy, and the house ...",33332,Hounslow,Beautiful Ensuite Richmond-upon-Thames borough,Private room,65.0
...,...,...,...,...,...,...
54329,[Fantastic hospitality],47948833,Newham,Cozy & quiet room,Private room,35.0
54330,[Salahudin was a great host! He was extremely ...,47957224,Westminster,Titiwangsa One Paddington,Private room,85.0
54331,"[Great place, very pleasent, great host, Cosy ...",47975947,Lambeth,2 Bedroom Flat in Clapham / Stockwell,Entire home/apt,75.0
54332,"[Easy check in, great place and location, the ...",47979693,Tower Hamlets,Luxury waterside apartment Canary Wharf,Entire home/apt,140.0


In [ ]:
#cleaned df. to csv('processed airbnb.csv')

In [72]:
#loading the dataset of positive text and Negative text file:
P_file = open ("positive.txt",'r')
positive_list = []
for line in P_file:
    stripped_line = line.strip()
    positive_list.append(stripped_line)
P_file. close ()
#Loading the dataset for negative text file
n_file = open("negative.txt", "r")
negative_list = []
for line in n_file:
    stripped_line = line.strip()
    negative_list.append(stripped_line)
n_file. close()
def positive_negative_checker(cleanW):
    print (cleanW)
#quit ()
# comparing whether the comment is negative or neutral or positive
    positive_count = 0;
    negative_count = 0;
    for c in cleanW:
        if c in positive_list:
            positive_count = positive_count + 1
        elif c in negative_list:
            negative_count = negative_count + 1
# print("Positive Count is: % In" %( positive_count))
# print "Negative Count is: %d In" %( negative_count))
#quit ()
    if negative_count > positive_count:
        return-1
    elif positive_count > negative_count:
        return 1
    else:
        return 0

In [73]:
import nltk
#stopwords = set(STOPWORDS) # STOPWORDS is a list with english common words that you should not count in the
# wordcloud, Like prepositions and conjuctions
# nltk.download('stopwords")
# nltk. download ('punkt")
# from nltk.corpus import stopwords
# stopWords = set(stopwords.words('english'))
#stopwords = set (STOPWORDS) # STOPWORDS is a List with english common words that you should not count in the
# wordcloud, Like prepositions and conjuctions

In [ ]:
#We are defining the function getRanks that receive only one parameter: the class to analyse.
#import word tokenize import collections,string from nltk import word tokenize def comment_analyser (sentence_list):
positive_comment_count = o
neutral_comment_count = 0
negative_comment_count = 0
for i in range(len (sentence_list)):
    words = word_tokenize(sentence_list[i]) #Tokenise all values stored in revtextC
words= w.lower() for w in words] #Change to Lower
words = word for word in words if word.isalnum()] # Remove conjuctions/punctuation
#words = [ele for ele in words if ele not in stopWords] #Remove stopwords
cleanW = [ele for ele in words if ele not in stopwords
comment_value = positive_negative_checker (cleanW)
if comment value == 1:
    positive_comment_count += 1 elif comment value == 0:
neutral_comment_count += 1 elif comment value == -1:
negative comment count += 1
analyzed_comment = [positive_comment_count, neutral_comment_count, negative_comment_count]
print (analyzed comment)
return analyzed_comment

In [ ]:
cleaned_df.insert (1, 'positive_comment', 0)
cleaned_df.insert (2, 'neutral_comment', 0) cleaned_df. insert (2, 'negative_comment'
                                                                , 0)

In [ ]:
cleaned df

In [ ]:
from wordcloud import Wordcloud, STOPWORDS
stopwords = set(STOPWORDS)
count = 0
checking_value = 0
for i in range(len (cleaned df)):
    checking_value = [] #gets the total positive comments, neutral comments and negative comments count
checking value = comment analyser (cleaned_df.iloc[i,0])
cleaned_df.iloc[i, 1] = checking_value[0] #put the total_positive_comment into positive_comment column
cleaned_df.iloc[i, 2] = checking_value[1] #put the total_neutral_comment into neutral_comment column
cleaned_df.iloc[i, 3] = checking_value [2] #put the total_negative_comment into negative_comment column
print(it1) # printing the total rows that have been proceeded

In [ ]:
#you can save the cleaned data as cs file using .to csv)
cleaned_df.to_csv('processed_airbnb.csv')

In [ ]:
cleaned_df